# Starting with...

In [28]:
import tweepy
import os
import time
import pandas as pd
from IPython.display import display, clear_output

In [3]:
# Defining our secret keys c:
consumer_key = os.environ['TWITTER_CONSUMER_KEY']
consumer_secret = os.environ['TWITTER_SECRET_KEY']
bearer_token = os.environ['TWITTER_BEARER_TOKEN']
access_token = os.environ['ACCESS_TOKEN']
access_token_secret = os.environ['ACCESS_TOKEN_SECRET']

In [4]:
# Getting authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

In [5]:
# Getting my authorization token
try:
    redirect_url = auth.get_authorization_url()
except tweepy.TweepError:
    print('Error! Failed to get request token.')

In [6]:
# Setting the access tokens
auth.set_access_token(access_token, access_token_secret)

In [7]:
# Now that we have access by this weird authenthification we can call the api for our app
api = tweepy.API(auth, wait_on_rate_limit=True)

In [8]:
# Downloading my home timeline tweets and printing one fo their texts.
public_tweets = api.home_timeline()
for tweet in public_tweets:
    #print(tweet.text) # Just uncomment this
    pass

In [9]:
# Get the User object for me c:...
user = api.get_user('DBubu73')

In [10]:
# Printing some information with 
print(user.screen_name)
print(user.friends_count)
for friend in user.friends():
   # print(friend.screen_name) #Just uncomments this
    pass

DBubu73
123


# Cursor

In [11]:
# Scrapping a user's first 200 tweets
username = 'selenagomez'
count = 200
try:     
    # Creation of query method using parameters
    tweets = tweepy.Cursor(api.user_timeline,id=username, tweet_mode="extended").items(count)

    # Pulling information from tweets iterable object
    tweets_list = [[tweet.created_at, tweet.id, tweet.full_text] for tweet in tweets]

    # Creation of dataframe from tweets list
    tweets_df = pd.DataFrame(tweets_list, columns=['Created', 'ID', 'Text'])
except BaseException as e:
    print('failed on_status,',str(e))
    time.sleep(3)

In [12]:
# TADAA
tweets_df

,Created,ID,Text
0,2021-07-27 14:28:18,1420028274655309837,"RT @OnlyMurdersHulu: Great, as if we weren’t a..."
1,2021-07-27 14:12:45,1420024364725121036,"We’re all neighbors here, but if I were you, I..."
2,2021-07-23 17:09:02,1418619174096564224,"RT @gofundme: Happy Birthday, @selenagomez 🥳\n..."
3,2021-07-22 18:56:07,1418283735812743168,RT @SelenaFanClub: We have a little gift for y...
4,2021-07-22 18:23:47,1418275595905998858,RT @rarebeauty: In honor of our founder’s birt...
...,...,...,...
195,2021-01-29 04:46:15,1355014370066128896,RT @rauwalejandro: Ya falta poco ... 😬
196,2021-01-29 04:46:12,1355014356929536001,@rauwalejandro 💃🏻
197,2021-01-29 04:08:49,1355004951680987136,RT @Tainy: The video for ‘Baila Conmigo’ is ou...
198,2021-01-29 04:08:32,1355004876716183553,RT @SteveMartinToGo: A special day on the set ...


In [13]:
# Notice that if is a retweet, it could be truncated :c
tweets_df['Text'][0]

'RT @OnlyMurdersHulu: Great, as if we weren’t already dealing with enough here, my “lovely” neighbors have decided to solve the murder in th…'

# Streaming With Tweepy

In [14]:
# Override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):

    def on_status(self, status):
        print(status.text)
    
    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_error disconnects the stream
            print("I disconnected bro")
            return False

        # returning non-False reconnects the stream, with backoff.

In [15]:
# Creating our stream Object
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)

In [ ]:
# Filtering all tweets containing certain word
word = 'python'
myStream.filter(track=[word])

In [ ]:
# Filtering all tweets from a user with the twitter ID
ID = str(user.id)
myStream.filter(follow=[ID])

In [18]:
# Disconnecting the stream
myStream.disconnect()

# Now we are going to scrape the first 100 tweets from some Rodolfo's Followers c:

In [ ]:
username = 'FerroRodolfo'
# Defining some auxiliar variables
count = 100
tweets_dfs = {}
maxi = 100

# Getting the id for every Rodo's follower.
followers = api.followers_ids(screen_name=username)
# Iterating for every follower
for index, follower in enumerate(followers[:maxi]):
    try:
        # Getting the user name
        user = api.get_user(follower)
        # Getting their tweets
        tweets = tweepy.Cursor(api.user_timeline,id=str(follower), tweet_mode="extended").items(count)
        # Just formatting
        tweets_dfs[user.screen_name] = [[tweet.created_at, tweet.id, tweet.full_text] for tweet in tweets]
        # DataFraming
        tweets_dfs[user.screen_name] = pd.DataFrame(tweets_dfs[user.screen_name], columns=['Created', 'ID', 'Text'])  
    except:
        # Exist some accounts that we can't access to.
        #print(f"User {follower} not found")
        pass
    # Process Counter
    print(f"{index}/{maxi-1}")
    clear_output(wait=True)

In [33]:
# Saving these tweets in a csv file
for user, tweets in tweets_dfs.items():
    tweets.to_csv(f'tweets/{user}_tweets.csv', header=False, index=False)

# The next part is applying NLP to know what the tweet is talking about and classifie them